Adapted from [ner_with_bilstm_and_crf](https://www.kaggle.com/nikkisharma536/ner-with-bilstm-and-crf/notebook)
Altigran Soares da Silva
IComp/UFAM - 15/03/2021


In [1]:
# Import libs
# Also remember to use GPU in your colab notebook
%tensorflow_version 2.x

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import math
from math import nan
import random
import json
from future.utils import iteritems
import pickle
from requests import get

# Needed for using CRF

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
# Code to read csv file from google drive into Colaboratory:
DATA_TRAINING_FILE_ID = '1hIe1NaUcYENwTFrG_z1WUEiEkEzhtzbx'
DATA_TRAINING_FILENAME = 'original_data.csv'
# DATA_DEV_FILE_ID = '1fB1Mp-6_zxGC3NO5sMX23B-LPyYzc9I9'
# DATA_DEV_FILENAME = 'ner_medical_dev.csv'
# DATA_TEST_FILE_ID = '17vmu2VXPwvOKuQ66kDY7t38dFqkYccvx'
# DATA_TEST_FILENAME = 'ner_medical_test.csv'
BACKUP_FOLDER_ID = '1YWR4Ip8w94RwFMyMtNpRa9M0FpiJtqd5'

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded_training = drive.CreateFile({'id': DATA_TRAINING_FILE_ID})
downloaded_training.GetContentFile(DATA_TRAINING_FILENAME)
# downloaded_dev = drive.CreateFile({'id': DATA_DEV_FILE_ID})
# downloaded_dev.GetContentFile(DATA_DEV_FILENAME)
# downloaded_test = drive.CreateFile({'id': DATA_TEST_FILE_ID})
# downloaded_test.GetContentFile(DATA_TEST_FILENAME)

# Read the csv file in a dataframe called "data"
training_data = pd.read_csv(DATA_TRAINING_FILENAME, encoding="utf-8")
# dev_data = pd.read_csv(DATA_DEV_FILENAME, encoding="latin1")
# test_data = pd.read_csv(DATA_TEST_FILENAME, encoding="latin1")
# Fill NaN values using the specified method
# Ffill propagate last valid observation/value forward to next valid 
training_data = training_data.fillna(method="ffill")
# dev_data = dev_data.fillna(method="ffill")
# test_data = test_data.fillna(method="ffill")

notebook_filename = get('http://172.28.0.2:9000/api/sessions').json()[0]['name']

In [3]:
new_df = pd.DataFrame({"A": training_data.groupby(['Sentence #'], as_index=False).agg({'Word': '||&|$||'.join})["Sentence #"],
                       "B": training_data.groupby(['Sentence #'], as_index=False).agg({'Word': '||&|$||'.join})["Word"],
                       "C": training_data.groupby(['Sentence #'], as_index=False).agg({'Tag': '||&|$||'.join})["Tag"]
})
new_df.head()

,A,B,C
0,Sentence: 1,Exhibit||&|$||99.1||&|$||JOINT||&|$||FILING||&...,O||&|$||O||&|$||B-DOC_NAME||&|$||I-DOC_NAME||&...
1,Sentence: 10,SERVICES||&|$||AGREEMENT||&|$||CONQUER||&|$||R...,B-DOC_NAME||&|$||I-DOC_NAME||&|$||O||&|$||O||&...
2,Sentence: 100,EXHIBIT||&|$||B||&|$||COOPERATION||&|$||AGREEM...,O||&|$||O||&|$||B-DOC_NAME||&|$||I-DOC_NAME||&...
3,Sentence: 101,Exhibit||&|$||10.18||&|$||MAINTENANCE||&|$||AG...,O||&|$||O||&|$||B-DOC_NAME||&|$||I-DOC_NAME||&...
4,Sentence: 102,Execution||&|$||Copy||&|$||CHARTER||&|$||AFFIL...,O||&|$||O||&|$||B-DOC_NAME||&|$||I-DOC_NAME||&...


In [4]:
df_train, df_val = train_test_split(new_df, test_size=0.15)

In [5]:
df_train["B"] = df_train["B"].apply(lambda x: x.split("||&|$||"))
df_train["C"] = df_train["C"].apply(lambda x: x.split("||&|$||"))
df_train.head()

df_train = df_train.explode(list('BC'))

df_val["B"] = df_val["B"].apply(lambda x: x.split("||&|$||"))
df_val["C"] = df_val["C"].apply(lambda x: x.split("||&|$||"))
df_val.head()

df_val = df_val.explode(list('BC'))

In [6]:
df_train = df_train.rename(columns = {"A":"Sentence #", "B": "Word", "C": "Tag"})
df_val = df_val.rename(columns = {"A":"Sentence #", "B": "Word", "C": "Tag"})

df_train.to_csv('ner_training_dataset.csv', index=False)
df_val.to_csv('ner_validation_dataset.csv', index=False)